In [8]:
# -*- coding: utf-8 -*-
"""
NiftyDataMining.py

This is script to load the NSE data, apply technical indicators
Moreover, it fetches data of other indices using multiple predefined CSVs and create a single data frame
"""
import pandas as pd
import gc
from talib import RSI, BBANDS, MACD, EMA, STOCH, ADX
import matplotlib.pyplot as plt
from nsepy import get_history
from datetime import date

In [16]:
# Read the CSV file into a DataFrame
df = pd.read_csv("reversed_nifty_data.csv")

#this is reference start date before EMA 200 days will fetch null
ref_startdate = date(2009,4,8)

#actual start date and end date from where the Nifty model to be built
startdate = date(2009, 11, 25)
enddate = date(2024,4,3)

# Display the first few rows of the DataFrame
print(df.head(2))


         Date    Price     Open     High      Low    Close
0  2009-04-08  3342.95  3255.35  3357.05  3149.25  3342.95
1  2009-04-09  3342.05  3346.00  3401.15  3307.05  3342.05


In [18]:
# Set 'Date' column as the index
df.set_index('Date', inplace=True)

# Calculating technical indicators such as EMA-200, EMA-100, EMA-50, EMA-21, EMA-5, MACD, RSI-14, Bollinger Bands
EMA_200 = EMA(df['Close'], 200)
EMA_100 = EMA(df['Close'], 100)
EMA_50 = EMA(df['Close'], 50)
EMA_21 = EMA(df['Close'], 21)
EMA_5 = EMA(df['Close'], 5)
df['EMA-200'] = EMA_200
df['EMA-100'] = EMA_100
df['EMA-50'] = EMA_50
df['EMA-21'] = EMA_21
df['EMA-5'] = EMA_5
df['MACD'], df['macdEMA'], df['macdHistory'] = MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['RSI-14'] = RSI(df['Close'], timeperiod=14)
df['BollingerUpperBand'], df['BollingerMiddleBand'], df['BollingerLowerBand'] = BBANDS(df['Close'], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)

# Print the DataFrame
print(df.tail(2))


               Price     Open     High       Low     Close       EMA-200  \
Date                                                                       
2024-04-02  22453.30  22458.8  22497.6  22388.15  22453.30  20605.261750   
2024-04-03  22434.65  22385.7  22521.1  22346.50  22434.65  20623.464618   

                 EMA-100        EMA-50        EMA-21         EMA-5       MACD  \
Date                                                                            
2024-04-02  21449.743792  21951.494102  22164.536900  22316.863088  71.143083   
2024-04-03  21469.246885  21970.441392  22189.092637  22356.125392  83.832115   

              macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
Date                                                                
2024-04-02  45.987117    25.155966  60.592400        22637.888451   
2024-04-03  53.556116    30.275999  59.854996        22615.659613   

            BollingerMiddleBand  BollingerLowerBand  
Date                                   

In [19]:
df.index = (pd.to_datetime(df.index))

df= df[startdate:]
#Importing Brent Crude price Data from CSV"""
df1=pd.read_csv("BrentCrude.csv")
print(df1.head(2))

         Date  Price
0  2024-04-02  89.29
1  2024-04-01  87.42


In [21]:
# Convert column name Date from String datatype to Date datatype"""
df1['Date']=pd.to_datetime(df1['Date'])

#Sort the Sheet based on Date ascending order. This is one of pre-requisite to merge different dataframe.
#Dataframe that has to be merged should be sorted by join columns. In this case, join column is date.
#Hence, it is sorted based on Date column
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date   Price_x     Open     High       Low     Close       EMA-200  \
3559 2024-04-02  22453.30  22458.8  22497.6  22388.15  22453.30  20605.261750   
3560 2024-04-03  22434.65  22385.7  22521.1  22346.50  22434.65  20623.464618   

           EMA-100        EMA-50        EMA-21         EMA-5       MACD  \
3559  21449.743792  21951.494102  22164.536900  22316.863088  71.143083   
3560  21469.246885  21970.441392  22189.092637  22356.125392  83.832115   

        macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
3559  45.987117    25.155966  60.592400        22637.888451   
3560  53.556116    30.275999  59.854996        22615.659613   

      BollingerMiddleBand  BollingerLowerBand  Price_y  
3559             22274.11        21910.331549    89.29  
3560             22360.10        22104.540387    89.29  


In [22]:
# Importing Dow Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("DowHistorical.csv")
df1['Date']=pd.to_datetime(df1['Date'])
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date   Price_x     Open     High       Low     Close       EMA-200  \
3559 2024-04-02  22453.30  22458.8  22497.6  22388.15  22453.30  20605.261750   
3560 2024-04-03  22434.65  22385.7  22521.1  22346.50  22434.65  20623.464618   

           EMA-100        EMA-50        EMA-21         EMA-5       MACD  \
3559  21449.743792  21951.494102  22164.536900  22316.863088  71.143083   
3560  21469.246885  21970.441392  22189.092637  22356.125392  83.832115   

        macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
3559  45.987117    25.155966  60.592400        22637.888451   
3560  53.556116    30.275999  59.854996        22615.659613   

      BollingerMiddleBand  BollingerLowerBand  Price_y     Price  
3559             22274.11        21910.331549    89.29  39171.22  
3560             22360.10        22104.540387    89.29  39171.22  


In [23]:

print(df.tail(2))

           Date   Price_x     Open     High       Low     Close       EMA-200  \
3559 2024-04-02  22453.30  22458.8  22497.6  22388.15  22453.30  20605.261750   
3560 2024-04-03  22434.65  22385.7  22521.1  22346.50  22434.65  20623.464618   

           EMA-100        EMA-50        EMA-21         EMA-5       MACD  \
3559  21449.743792  21951.494102  22164.536900  22316.863088  71.143083   
3560  21469.246885  21970.441392  22189.092637  22356.125392  83.832115   

        macdEMA  macdHistory     RSI-14  BollingerUpperBand  \
3559  45.987117    25.155966  60.592400        22637.888451   
3560  53.556116    30.275999  59.854996        22615.659613   

      BollingerMiddleBand  BollingerLowerBand  Price_y     Price  
3559             22274.11        21910.331549    89.29  39171.22  
3560             22360.10        22104.540387    89.29  39171.22  


In [27]:
""" Importing FTSE Historical price Data from CSV. And perform similar action like BrentCrude price data"""
df1=pd.read_csv("FTSE_Historical.csv")
df1['Date']=pd.to_datetime(df1['Date'])
df1=df1.sort_values(by=['Date'],ascending=True)
df=pd.merge_asof(df,df1,on='Date')
print(df.tail(2))

           Date   Price_x     Open     High       Low     Close       EMA-200  \
3559 2024-04-02  22453.30  22458.8  22497.6  22388.15  22453.30  20605.261750   
3560 2024-04-03  22434.65  22385.7  22521.1  22346.50  22434.65  20623.464618   

           EMA-100        EMA-50        EMA-21  ...       MACD    macdEMA  \
3559  21449.743792  21951.494102  22164.536900  ...  71.143083  45.987117   
3560  21469.246885  21970.441392  22189.092637  ...  83.832115  53.556116   

      macdHistory     RSI-14  BollingerUpperBand  BollingerMiddleBand  \
3559    25.155966  60.592400        22637.888451             22274.11   
3560    30.275999  59.854996        22615.659613             22360.10   

      BollingerLowerBand  Price_y     Price     FTSE  
3559        21910.331549    89.29  39171.22  7935.09  
3560        22104.540387    89.29  39171.22  7937.44  

[2 rows x 21 columns]
